In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

C:\Users\colli\AppData\Local\Programs\Python\Python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### FUNCTIONS

In [19]:
def get_data(drop_8000=True):
    df = pd.read_csv('../raw_data/wra_CT_PM_conclusions.csv')
    df.drop(columns='Unnamed: 0', inplace=True)
    if drop_8000:
        df = df[df['n_trials'] < 8000]
    df.conclusions.fillna('',inplace=True)
    #df['therapeutic_number'] = df['Therapeutic area'].apply(lambda x: x.count(',') + 1)
    return df

In [20]:
def get_numericals(df):
    df['therapeutic_number'] = df['Therapeutic area'].apply(lambda x: x.count(',') + 1)
    df = df.select_dtypes(exclude='object')
    return df

In [21]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def clean(x):
    for punctuation in string.punctuation:
        x = x.replace(punctuation, '')
    x = x.lower()
    x = ''.join(word for word in x if not word.isdigit())
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(x)
    x = [w for w in word_tokens if not w in stop_words]
    x = ' '.join(x)
    return x

### GET AND CLEAN DATA

In [22]:
df = get_data()

In [23]:
numerical = get_numericals(df)

In [24]:
df['abstracts'] = df.conclusions.apply(clean)

In [25]:
model_df = pd.concat([numerical, df.abstracts],axis=1)

### X AND Y, SCALE AND SPLIT

In [26]:
X = model_df.drop(columns='Authorisation status')
y = model_df['Authorisation status']

In [27]:
X_text = X.abstracts
X_num = X.drop(columns='abstracts')

In [28]:
scaler = RobustScaler()
X_numbers = scaler.fit_transform(X_num) # bit naughty here - i should fit and transform on the train and transform only the test

In [29]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),max_features=50, max_df=0.8, min_df=0.05, sublinear_tf=True)
X_text_vect = vectorizer.fit_transform(X_text)
X_vectors = X_text_vect.toarray()

In [30]:
X_combined = np.concatenate((X_numbers,X_vectors),axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=1)

### MODEL

In [34]:
forest = RandomForestClassifier(random_state=0,
                                bootstrap=False,
                                max_depth=None,
                                min_samples_leaf=3,
                                min_samples_split=5,
                                n_estimators=34)

----